In [6]:
# ===================================================================
# الخلية 1: الإعداد وتحميل البيانات
# ===================================================================

# تثبيت المكتبات
!pip install -q kaggle dask[complete]

# استيراد المكتبات
import os, time, gzip, shutil, glob, json
import pandas as pd
import numpy as np
import dask.dataframe as dd
from google.colab import files

print("="*60)
print("📦 TP02: التعامل مع ملفات البيانات الضخمة (5GB+)")
print("="*60)

# ====== إعداد Kaggle API ======
print("\n🔐 إعداد Kaggle API...")
print("📁 الرجاء رفع ملف kaggle.json:")
uploaded = files.upload()

uploaded_filename = list(uploaded.keys())[0]
if uploaded_filename != 'kaggle.json':
    os.rename(uploaded_filename, 'kaggle.json')

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("✅ تم إعداد Kaggle API")

# ====== تحميل Dataset ======
DATASET = "mkechinov/ecommerce-behavior-data-from-multi-category-store"
print(f"\n⏳ جاري تحميل: {DATASET}")
!kaggle datasets download -d {DATASET}

# فك الضغط
print("\n⏳ جاري فك الضغط...")
zip_file = glob.glob("*.zip")[0]
!unzip -q {zip_file}

# تحديد الملف
DATA_CSV = "2019-Oct.csv"
file_size_gb = os.path.getsize(DATA_CSV) / (1024**3)
print(f"\n✅ جاهز! الملف: {DATA_CSV} ({file_size_gb:.2f} GB)")
!head -3 {DATA_CSV}

# ====== وظائف مساعدة ======
def get_file_size_mb(path):
    return os.path.getsize(path) / (1024**2) if os.path.exists(path) else None

def measure_time(func, *args, **kwargs):
    start = time.time()
    result = func(*args, **kwargs)
    return result, time.time() - start

print("\n✅ الإعداد مكتمل!")


📦 TP02: التعامل مع ملفات البيانات الضخمة (5GB+)

🔐 إعداد Kaggle API...
📁 الرجاء رفع ملف kaggle.json:


Saving kaggle(1).json to kaggle(1).json
✅ تم إعداد Kaggle API

⏳ جاري تحميل: mkechinov/ecommerce-behavior-data-from-multi-category-store
Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
100% 4.27G/4.29G [00:51<00:00, 252MB/s]
100% 4.29G/4.29G [00:52<00:00, 88.5MB/s]

⏳ جاري فك الضغط...

✅ جاهز! الملف: 2019-Oct.csv (5.28 GB)
event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc

✅ الإعداد مكتمل!


In [7]:
# ===================================================================
# الخلية 2: تطبيق الطرق الثلاث والمقارنة
# ===================================================================

results = []

print("="*60)
print("🔹 بدء تنفيذ الطرق الثلاث")
print("="*60)

# ========== الطريقة 1: Pandas Chunks ==========
print("\n⏳ الطريقة 1: Pandas Chunks...")

def method_pandas_chunks(file_path, chunksize=100000):
    total = 0
    for chunk in pd.read_csv(file_path, chunksize=chunksize, low_memory=False):
        total += len(chunk)
    return total

rows_pandas, time_pandas = measure_time(method_pandas_chunks, DATA_CSV)
print(f"✅ Pandas: {rows_pandas:,} صف في {time_pandas:.2f}s ({rows_pandas/time_pandas:,.0f} صف/ثانية)")

results.append({
    'method': 'Pandas Chunks',
    'rows': rows_pandas,
    'time_seconds': round(time_pandas, 2),
    'speed': round(rows_pandas/time_pandas, 0)
})

# ========== الطريقة 2: Dask ==========
print("\n⏳ الطريقة 2: Dask...")

def method_dask(file_path):
    ddf = dd.read_csv(file_path, blocksize="64MB", assume_missing=True)
    return ddf.shape[0].compute()

rows_dask, time_dask = measure_time(method_dask, DATA_CSV)
print(f"✅ Dask: {rows_dask:,} صف في {time_dask:.2f}s ({rows_dask/time_dask:,.0f} صف/ثانية)")

results.append({
    'method': 'Dask',
    'rows': rows_dask,
    'time_seconds': round(time_dask, 2),
    'speed': round(rows_dask/time_dask, 0)
})

# ========== الطريقة 3: Compression ==========
print("\n⏳ الطريقة 3: Compression...")

COMPRESSED_FILE = DATA_CSV + ".gz"

# ضغط الملف
def compress_file(inp, out):
    with open(inp, 'rb') as f_in, gzip.open(out, 'wb', compresslevel=6) as f_out:
        shutil.copyfileobj(f_in, f_out)

_, time_compress = measure_time(compress_file, DATA_CSV, COMPRESSED_FILE)

original_mb = get_file_size_mb(DATA_CSV)
compressed_mb = get_file_size_mb(COMPRESSED_FILE)
compression_ratio = ((original_mb - compressed_mb) / original_mb) * 100

print(f"   ضغط: {original_mb:.2f} MB → {compressed_mb:.2f} MB ({compression_ratio:.1f}% توفير) في {time_compress:.2f}s")

# قراءة الملف المضغوط
def method_compressed(file_path, chunksize=100000):
    total = 0
    for chunk in pd.read_csv(file_path, compression='gzip', chunksize=chunksize, low_memory=False):
        total += len(chunk)
    return total

rows_comp, time_read = measure_time(method_compressed, COMPRESSED_FILE)
total_time = time_compress + time_read

print(f"✅ Compression: {rows_comp:,} صف في {total_time:.2f}s (ضغط: {time_compress:.2f}s + قراءة: {time_read:.2f}s)")

results.append({
    'method': 'Compression',
    'rows': rows_comp,
    'time_seconds': round(total_time, 2),
    'speed': round(rows_comp/total_time, 0),
    'original_mb': round(original_mb, 2),
    'compressed_mb': round(compressed_mb, 2),
    'compression_ratio': round(compression_ratio, 2)
})

# ========== حفظ النتائج ==========
results_df = pd.DataFrame(results)
results_df.to_csv('comparison_results.csv', index=False)

print("\n" + "="*60)
print("📊 مقارنة النتائج")
print("="*60)
print(results_df[['method', 'rows', 'time_seconds', 'speed']].to_string(index=False))

print("\n💾 مقارنة المساحة:")
print(f"   الأصلي: {original_mb:.2f} MB")
print(f"   المضغوط: {compressed_mb:.2f} MB")
print(f"   التوفير: {compression_ratio:.1f}%")

fastest = results_df.loc[results_df['time_seconds'].idxmin(), 'method']
print(f"\n🏆 أسرع طريقة: {fastest}")


🔹 بدء تنفيذ الطرق الثلاث

⏳ الطريقة 1: Pandas Chunks...
✅ Pandas: 42,448,764 صف في 119.42s (355,469 صف/ثانية)

⏳ الطريقة 2: Dask...
✅ Dask: 42,448,764 صف في 113.44s (374,206 صف/ثانية)

⏳ الطريقة 3: Compression...
   ضغط: 5406.01 MB → 1661.23 MB (69.3% توفير) في 308.61s
✅ Compression: 42,448,764 صف في 462.03s (ضغط: 308.61s + قراءة: 153.42s)

📊 مقارنة النتائج
       method     rows  time_seconds    speed
Pandas Chunks 42448764        119.42 355469.0
         Dask 42448764        113.44 374206.0
  Compression 42448764        462.03  91875.0

💾 مقارنة المساحة:
   الأصلي: 5406.01 MB
   المضغوط: 1661.23 MB
   التوفير: 69.3%

🏆 أسرع طريقة: Dask


In [11]:
# ===================================================================
# الخلية 3: إنشاء التقرير وتحميل النتائج
# ===================================================================

print("="*60)
print("📄 إنشاء التقرير النهائي")
print("="*60)

# ====== إنشاء README.md ======
readme_text = f"""# TP02: التعامل مع ملفات البيانات الضخمة (5GB+)

## وصف المشروع
تطبيق ثلاث طرق للتعامل مع ملفات CSV الكبيرة في Python:
1. Pandas Chunks: قراءة على دفعات
2. Dask: حوسبة موزعة
3. Compression: ضغط + قراءة

## Dataset المستخدم
- المصدر: eCommerce Behavior Data
- الملف: {DATA_CSV}
- الحجم: {file_size_gb:.2f} GB
- الصفوف: {rows_pandas:,}

## النتائج

### مقارنة الأداء
| الطريقة | الوقت (ثانية) | السرعة (صف/ثانية) |
|---------|---------------|-------------------|
| Pandas Chunks | {time_pandas:.2f} | {rows_pandas/time_pandas:,.0f} |
| Dask | {time_dask:.2f} | {rows_dask/time_dask:,.0f} |
| Compression | {total_time:.2f} | {rows_comp/total_time:,.0f} |

### مقارنة المساحة
- الأصلي: {original_mb:,.2f} MB
- المضغوط: {compressed_mb:,.2f} MB
- التوفير: {compression_ratio:.1f}%

## الاستنتاجات
- أسرع طريقة: {fastest}
- الضغط يوفر {compression_ratio:.1f}% من المساحة
- كل طريقة مناسبة لسيناريو معين

## المتطلبات
pandas>=1.3.0
dask[complete]>=2021.10.0
kaggle>=1.5.0

## التاريخ
{pd.Timestamp.now().strftime('%Y-%m-%d')}
"""

with open('README.md', 'w', encoding='utf-8') as f:
    f.write(readme_text)

# ====== إنشاء requirements.txt ======
requirements_text = """pandas>=1.3.0
dask[complete]>=2021.10.0
kaggle>=1.5.0
numpy>=1.20.0
"""
with open('requirements.txt', 'w') as f:
    f.write(requirements_text)

print("✅ تم إنشاء README.md و requirements.txt")

# ====== عرض النتائج التفصيلية ======
print("\n" + "="*60)
print("📈 النتائج التفصيلية")
print("="*60)
print(results_df.to_string(index=False))

# ====== التحليل النهائي ======
print("\n" + "="*60)
print("🎯 التحليل والتوصيات")
print("="*60)

print(f"\n📊 مقارنة الأداء النسبي:")
print(f"   • Pandas: {time_pandas:.2f}s (الأساس)")
print(f"   • Dask: {time_dask:.2f}s ({(time_dask/time_pandas-1)*100:+.1f}%)")
print(f"   • Compression: {total_time:.2f}s ({(total_time/time_pandas-1)*100:+.1f}%)")

print(f"\n💾 توفير المساحة:")
print(f"   • التوفير: {original_mb - compressed_mb:,.2f} MB ({compression_ratio:.1f}%)")

print("\n✅ التوصيات:")
print("   1. للسرعة القصوى: استخدم أسرع طريقة")
print("   2. لتوفير المساحة: استخدم الضغط")
print("   3. للبيانات الضخمة جداً: استخدم Dask")
print("   4. للتطبيقات العامة: Pandas Chunks متوازنة")

📄 إنشاء التقرير النهائي
✅ تم إنشاء README.md و requirements.txt

📈 النتائج التفصيلية
       method     rows  time_seconds    speed  original_mb  compressed_mb  compression_ratio
Pandas Chunks 42448764        119.42 355469.0          NaN            NaN                NaN
         Dask 42448764        113.44 374206.0          NaN            NaN                NaN
  Compression 42448764        462.03  91875.0      5406.01        1661.23              69.27

🎯 التحليل والتوصيات

📊 مقارنة الأداء النسبي:
   • Pandas: 119.42s (الأساس)
   • Dask: 113.44s (-5.0%)
   • Compression: 462.03s (+286.9%)

💾 توفير المساحة:
   • التوفير: 3,744.78 MB (69.3%)

✅ التوصيات:
   1. للسرعة القصوى: استخدم أسرع طريقة
   2. لتوفير المساحة: استخدم الضغط
   3. للبيانات الضخمة جداً: استخدم Dask
   4. للتطبيقات العامة: Pandas Chunks متوازنة
